In [1]:
#######################################################################################################
# Summary
# 1. PyTorch Multi-GPU example
# 2. On-the-fly data-augmentation (random crop, random flip)
# Slightly rewritten for 0.4.0+ API
#######################################################################################################

In [2]:
MULTI_GPU = True

In [3]:
import os
import sys
import time
import multiprocessing
import numpy as np
import pandas as pd
import torch
import torchvision.models as models
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.nn.init as init
import torchvision.transforms as transforms
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.autograd import Variable
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics.ranking import roc_auc_score
from sklearn.model_selection import train_test_split
from PIL import Image
from common.utils import download_data_chextxray, get_imgloc_labels, get_train_valid_test_split
from common.utils import compute_roc_auc, get_cuda_version, get_cudnn_version, get_gpu_name
from common.utils import yield_mb
from common.params_dense import *

In [4]:
print("OS: ", sys.platform)
print("Python: ", sys.version)
print("PyTorch: ", torch.__version__)
print("Numpy: ", np.__version__)
print("GPU: ", get_gpu_name())
print(get_cuda_version())
print("CuDNN Version ", get_cudnn_version())

OS:  linux
Python:  3.5.4 |Anaconda custom (64-bit)| (default, Nov 20 2017, 18:44:38) 
[GCC 7.2.0]
PyTorch:  0.4.0
Numpy:  1.14.1
GPU:  ['Tesla V100-PCIE-16GB', 'Tesla V100-PCIE-16GB', 'Tesla V100-PCIE-16GB', 'Tesla V100-PCIE-16GB']
CUDA Version 9.0.176
CuDNN Version  7.0.5


In [5]:
CPU_COUNT = multiprocessing.cpu_count()
GPU_COUNT = len(get_gpu_name())
print("CPUs: ", CPU_COUNT)
print("GPUs: ", GPU_COUNT)

CPUs:  24
GPUs:  4


In [6]:
# Model-params
IMAGENET_RGB_MEAN_TORCH = [0.485, 0.456, 0.406]
IMAGENET_RGB_SD_TORCH = [0.229, 0.224, 0.225]
# Paths
CSV_DEST = "chestxray"
IMAGE_FOLDER = os.path.join(CSV_DEST, "images")
LABEL_FILE = os.path.join(CSV_DEST, "Data_Entry_2017.csv")
print(IMAGE_FOLDER, LABEL_FILE)

chestxray/images chestxray/Data_Entry_2017.csv


In [7]:
# Manually scale to multi-gpu
assert torch.cuda.is_available()
_DEVICE = torch.device("cuda:0")
# enables cudnn's auto-tuner
torch.backends.cudnn.benchmark=True
if MULTI_GPU:
    LR *= GPU_COUNT 
    BATCHSIZE *= GPU_COUNT

In [8]:
%%time
# Download data
# Wall time: 17min 58s
print("Please make sure to download")
print("https://docs.microsoft.com/en-us/azure/storage/common/storage-use-azcopy-linux#download-and-install-azcopy")
download_data_chextxray(CSV_DEST)

Please make sure to download
https://docs.microsoft.com/en-us/azure/storage/common/storage-use-azcopy-linux#download-and-install-azcopy
Data already exists
CPU times: user 559 ms, sys: 263 ms, total: 822 ms
Wall time: 822 ms


In [9]:
#####################################################################################################
## Data Loading

In [10]:
# Normalise by imagenet mean/sd
normalize = transforms.Normalize(IMAGENET_RGB_MEAN_TORCH,
                                 IMAGENET_RGB_SD_TORCH)

In [11]:
class XrayData(Dataset):
    def __init__(self, img_dir, lbl_file, patient_ids, transform=None):
        
        self.img_locs, self.labels = get_imgloc_labels(img_dir, lbl_file, patient_ids)
        self.transform = transform
        print("Loaded {} labels and {} images".format(len(self.labels), len(self.img_locs)))
    
    def __getitem__(self, idx):
        im_file = self.img_locs[idx]
        im_rgb = Image.open(im_file)
        label = self.labels[idx]
        if self.transform is not None:
            im_rgb = self.transform(im_rgb)
        return im_rgb, torch.FloatTensor(label)
        
    def __len__(self):
        return len(self.img_locs)

In [12]:
def no_augmentation_dataset(img_dir, lbl_file, patient_ids, normalize):
    dataset = XrayData(img_dir, lbl_file, patient_ids,
                       transform=transforms.Compose([
                           transforms.Resize(WIDTH),
                           transforms.ToTensor(),  
                           normalize]))
    return dataset

In [13]:
train_set, valid_set, test_set = get_train_valid_test_split(TOT_PATIENT_NUMBER)

train:21563 valid:3080 test:6162


In [14]:
# Dataset for training
train_dataset = XrayData(img_dir=IMAGE_FOLDER,
                         lbl_file=LABEL_FILE,
                         patient_ids=train_set,
                         transform=transforms.Compose([
                             transforms.RandomResizedCrop(size=WIDTH),
                             transforms.RandomHorizontalFlip(),
                             transforms.ToTensor(),  # need to convert image to tensor!
                             normalize]))

Loaded 87306 labels and 87306 images


In [15]:
valid_dataset = no_augmentation_dataset(IMAGE_FOLDER, LABEL_FILE, valid_set, normalize)
test_dataset = no_augmentation_dataset(IMAGE_FOLDER, LABEL_FILE, test_set, normalize)

Loaded 7616 labels and 7616 images
Loaded 17198 labels and 17198 images


In [16]:
#####################################################################################################
## Helper Functions

In [17]:
def get_symbol(out_features=CLASSES, multi_gpu=MULTI_GPU):
    model = models.densenet.densenet121(pretrained=True)
    # Replace classifier (FC-1000) with (FC-14)
    model.classifier = nn.Sequential(
        nn.Linear(model.classifier.in_features, out_features), 
        nn.Sigmoid())
    if multi_gpu:
        model = nn.DataParallel(model)
    # CUDA
    model.to(_DEVICE)  
    return model

In [18]:
def init_symbol(sym, lr=LR):
    # BCE Loss since classes not mutually exclusive + Sigmoid FC-layer
    cri = nn.BCELoss()
    opt = optim.Adam(sym.parameters(), lr=lr, betas=(0.9, 0.999))
    sch = ReduceLROnPlateau(opt, factor=0.1, patience=5, mode='min')
    return opt, cri, sch 

In [19]:
def train_epoch(model, dataloader, optimizer, criterion):
    model.train()
    print("Training epoch")
    loss_val = 0
    for i, (data, target) in enumerate(dataloader): 
        # Get samples (both async)
        data, target = data.cuda(non_blocking=True), target.cuda(non_blocking=True)
        # Forwards
        output = model(data)
        # Loss
        loss = criterion(output, target)
        # Back-prop
        optimizer.zero_grad()
        # Log the loss (before .backward())
        loss_val += loss.item()
        loss.backward()
        optimizer.step()   
    print("Training loss: {0:.4f}".format(loss_val/i))

In [20]:
def valid_epoch(model, dataloader, criterion, phase='valid', cl=CLASSES):
    model.eval()
    if phase == 'testing':
        print("Testing epoch")
    else:
        print("Validating epoch")
        
    # Don't save gradients
    with torch.no_grad():
        if phase == 'testing':
            # pre-allocate predictions
            len_pred = len(dataloader)*(dataloader.batch_size)
            num_lab = dataloader.dataset.labels.shape[-1]
            out_pred = torch.cuda.FloatTensor(len_pred, num_lab).fill_(0)
        loss_val = 0
        for i, (data, target) in enumerate(dataloader): 
            # Get samples
            data, target = data.cuda(non_blocking=True), target.cuda(non_blocking=True)
             # Forwards
            output = model(data)
            # Loss
            loss = criterion(output, target)
            # Log the loss
            loss_val += loss.item()
            # Log for AUC
            if phase == 'testing':
                out_pred[output.size(0)*i:output.size(0)*(1+i)] = output.data
        # Fina loss
        loss_mean = loss_val/i 
    
    if phase == 'testing':
        out_gt = dataloader.dataset.labels
        out_pred = out_pred.cpu().numpy()[:len(out_gt)]  # Trim padding
        print("Test-Dataset loss: {0:.4f}".format(loss_mean))
        print("Test-Dataset AUC: {0:.4f}".format(compute_roc_auc(out_gt, out_pred, cl)))
    else:
        print("Validation loss: {0:.4f}".format(loss_mean))
    return loss_mean

In [21]:
# DataLoaders
train_loader = DataLoader(dataset=train_dataset, batch_size=BATCHSIZE,
                          shuffle=True, num_workers=CPU_COUNT, pin_memory=True)
# Using a bigger batch-size (than BATCHSIZE) for below worsens performance
valid_loader = DataLoader(dataset=valid_dataset, batch_size=BATCHSIZE,
                          shuffle=False, num_workers=CPU_COUNT, pin_memory=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=BATCHSIZE,
                         shuffle=False, num_workers=CPU_COUNT, pin_memory=True)

In [22]:
#####################################################################################################
## Train CheXNet

In [23]:
%%time
# Load symbol
chexnet_sym = get_symbol()

/anaconda/envs/py35/lib/python3.5/site-packages/torchvision-0.2.1-py3.5.egg/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.


CPU times: user 4.43 s, sys: 1.38 s, total: 5.81 s
Wall time: 5.97 s


In [24]:
%%time
# Load optimiser, loss
# Scheduler for LRPlateau is not used
optimizer, criterion, scheduler = init_symbol(chexnet_sym)

CPU times: user 1.4 ms, sys: 367 µs, total: 1.77 ms
Wall time: 1.78 ms


In [25]:
%%time
# 1 GPU - Main training loop: 26min 50s
# 4 GPU - Main training loop: 10min 11s
# Main train/val loop
for j in range(EPOCHS):
    stime = time.time()
    train_epoch(chexnet_sym, train_loader, optimizer, criterion)
    loss_val = valid_epoch(chexnet_sym, valid_loader, criterion)   
    print("Epoch time: {0:.0f} seconds".format(time.time()-stime))

Training epoch
Training loss: 0.1762
Validating epoch
Validation loss: 0.1521
Epoch time: 143 seconds
Training epoch
Training loss: 0.1595
Validating epoch
Validation loss: 0.1477
Epoch time: 118 seconds
Training epoch
Training loss: 0.1567
Validating epoch
Validation loss: 0.1524
Epoch time: 117 seconds
Training epoch
Training loss: 0.1547
Validating epoch
Validation loss: 0.1486
Epoch time: 117 seconds
Training epoch
Training loss: 0.1538
Validating epoch
Validation loss: 0.1464
Epoch time: 118 seconds
CPU times: user 13min 31s, sys: 3min 7s, total: 16min 38s
Wall time: 10min 12s


In [26]:
#####################################################################################################
## Test CheXNet

In [27]:
%%time
# Load model for testing
# I comment this out to create a fair test against Keras
#chexnet_sym_test = get_symbol()
#chkpt = torch.load("best_chexnet.pth.tar")
#chexnet_sym_test.load_state_dict(chkpt['state_dict'])

CPU times: user 5 µs, sys: 2 µs, total: 7 µs
Wall time: 12.6 µs


In [28]:
%%time
# 1 GPU AUC: 0.8156
# 4 GPU AUC: 0.8079
#test_loss = valid_epoch(chexnet_sym_test, test_loader, criterion, 'testing')
test_loss = valid_epoch(chexnet_sym, test_loader, criterion, 'testing')

Testing epoch
Test-Dataset loss: 0.1556
Full AUC [0.8092255172711226, 0.8586477243456379, 0.7938521293476548, 0.8882328870732145, 0.884375140101196, 0.879147972235656, 0.7101321841248905, 0.8282131914207386, 0.63879993442208, 0.850428754607819, 0.7448329947086632, 0.7949218733719277, 0.7620031083092549, 0.8682531643185114]
Test-Dataset AUC: 0.8079
CPU times: user 13.8 s, sys: 7.61 s, total: 21.5 s
Wall time: 15.2 s


In [29]:
#####################################################################################################
## Synthetic Data (Pure Training)

In [30]:
# Test on fake-data -> no IO lag
batch_in_epoch = len(train_dataset.labels)//BATCHSIZE
tot_num = batch_in_epoch * BATCHSIZE
print(tot_num)

87296


In [31]:
fake_X = torch.tensor(np.random.rand(tot_num, 3, 224, 224).astype(np.float32))
fake_y = torch.tensor(np.random.rand(tot_num, CLASSES).astype(np.float32))

In [32]:
%%time
# 1 GPU - Synthetic data: 25min 14s
# 4 GPU - Synthetic data: 8min 27s
for j in range(EPOCHS):
    train_epoch(chexnet_sym, 
                yield_mb(fake_X, fake_y, BATCHSIZE, shuffle=False),
                optimizer, 
                criterion)

Training epoch
Training loss: 0.7157
Training epoch
Training loss: 0.6954
Training epoch
Training loss: 0.6954
Training epoch
Training loss: 0.6954
Training epoch
Training loss: 0.6955
CPU times: user 12min 2s, sys: 1min 28s, total: 13min 31s
Wall time: 8min 32s
